# Iris Recognition for Biometrics



## Background:
Normal human vision consists of a series of rapid eye movements called saccades, which exhibit ballistic and abrupt motion behavior. By tracking optokinetic motion with displayed stimulus patterns on a screen, a person's visual performance can be measured from their gaze. Eye trackers are used in visual system research, psychology, psycholinguistics, marketing, and even to detect drowsiness in drivers.

## The problem:
Capturing and processing biodata in the real world is noisy and tedious. Describe a DL /AI architecture, algorithm, framework, bioinformatics approach, protocol, method, or approach you would take to process and extract human gaze from a camera

## Proposed Solution: 
For this problem, I will be using the Daugman Algorithm so that I will be able to detect the iris of an eye.

In [19]:
## imports

import numpy as np
import cv2
import itertools
import math


In [263]:
def daugman(grey_image, center, min_radius, max_radius, step):
    
    intensities = []
    list_of_radiuses = list(range(min_radius, max_radius, step)) # list of radius rings we want to analyse
    mask = np.zeros_like(grey_image)
    for radius in list_of_radiuses:
        # drawing a circle on the mask
        cv2.circle(mask, center, radius, 255, 1)
        
        # extracting out the ring
        contrast = grey_image & mask
        
        total_intensity = np.sum(contrast.flatten())
        average_intensity = total_intensity / (2 * math.pi * radius) # finding the average intensity
        intensities.append(average_intensity)

        mask = np.zeros_like(grey_image) # restart mask
    
    
    intensities = np.array(intensities)
    
    # finding the difference between each intensity ring
    intensities_diff = np.diff(intensities)
    
    intensities_diff = abs(cv2.GaussianBlur(intensities_diff, (1, 5), 0))


    index = np.argmax(intensities_diff) # finding the index of the maximum difference in intensity
    
    return intensities_diff[index], list_of_radiuses[index]
        
        
        
    
    

In [264]:
def iris(grey_image, min_radius, max_radius, step_coords, step_radius):
    
    height, width = grey_image.shape
    if height != width:
        # crop image into a square
        if height < width:
            grey_image = grey_image[:, int((width-height)/2): width-int((width-height)/2)-1]
        else:
            grey_image = grey_image[int((height-width)/2): height-int((height-width)/2)-1,:]
        
    height, width = grey_image.shape
        
    # let us assume that the iris is in the centre 1/3 of the image
    
    # list of coordinates that are inside the centre 1/3 of the image...
    one_third_x = list(range(int(height/3), int(height/3 * 2), step_coords))
    centre_third_square = itertools.product(one_third_x, one_third_x) # collecting coordinates in the centre 1/3
    intensity = []
    coords = []
    
    for pt in centre_third_square:
        intenseness, radius = daugman(grey_image, pt, min_radius, max_radius, step_radius)
        intensity.append(intenseness)
        coords.append((pt, radius))
        
    
    intensity = np.array(intensity)
    max_index = np.argmax(intensity)

    return coords[max_index]
    
    

In [265]:
image = cv2.imread('eye2.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
daugman(image, (5, 5), 2, 10, 3)




(array([19.09859317]), 2)

In [266]:
iris(image, 15, 100, 1, 3)
    
    

((58, 57), 36)